In [1]:
import algorithms
import data_clean
import pandas as pd
import numpy as np

In [2]:
df = data_clean.do(data_clean)
df = algorithms.ml_probabilities(algorithms,df)
df_historic_win_data = data_clean.clean_data("spreadspoke_scores.csv", "nfl_teams.csv", "1990-9-10") #winners by spread since
win_dict = algorithms.win_percent(df_historic_win_data)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [3]:
df[(df['spread_favorite'] == (-1 * 26.5))]                

,schedule_date,home,score_home,away,score_away,team_favorite_id,spread_favorite,home_ML,away_ML,implied_win_home,implied_win_away,total_implied_prob
1149,2013-10-13,DEN,35.0,JAX,19.0,DEN,-26.5,-6732.5,2419.0,0.985364,0.039698,1.025062


In [4]:
win_dict

{-1.0: 0.502762,
 -1.5: 0.505792,
 -2.0: 0.590747,
 -2.5: 0.578544,
 -3.0: 0.678171,
 -3.5: 0.741325,
 -4.0: 0.762295,
 -4.5: 0.784983,
 -5.0: 0.828054,
 -5.5: 0.837456,
 -6.0: 0.845946,
 -6.5: 0.848635,
 -7.0: 0.885362,
 -7.5: 0.926923,
 -8.0: 0.883721,
 -8.5: 0.918919,
 -9.0: 0.923913,
 -9.5: 0.9625,
 -10.0: 0.947115,
 -10.5: 0.969925,
 -11.0: 1.0,
 -11.5: 0.947368,
 -12.0: 0.956522,
 -12.5: 0.93617,
 -13.0: 0.975,
 -13.5: 0.968085,
 -14.0: 0.986486,
 -14.5: 0.977778,
 -15.0: 1.0,
 -15.5: 1.0,
 -16.0: 1.0,
 -16.5: 0.944444,
 -17.0: 1.0,
 -17.5: 1.0,
 -18.0: 1.0,
 -18.5: 1.0,
 -19.0: 1.0,
 -19.5: 1.0,
 -20.0: 1.0,
 -20.5: 1.0,
 -21.0: 1.0,
 -21.5: 1.0,
 -22.5: 1.0,
 -24.0: 1.0,
 -24.5: 1.0,
 -26.5: 1.0}

In [5]:
x = -3
underdog_implied_ml = 0.444444
round(((1 - win_dict[x]) * underdog_implied_ml) - (win_dict[x]),6)

-0.535136

In [6]:
x = -3.5
implied_ml = .658120
round((win_dict[x] * implied_ml) - (1 - win_dict[x]),6)


0.229206

In [7]:
#need to still incorporate epsilon rate -> will need to run montecarlo to determine
def plus_ev_games(win_dict, df, min_ev, largest_spread_fav=-100): #spread is always in negative
    #defeault largest spread fav and unlimited
    plus_ev = pd.DataFrame()
    for i,row in df.iterrows(): #iterate through each row in the dataframe
        spread = row.spread_favorite
        implied_ml = 0.01
        if row.team_favorite_id == row.home:
            fav_implied_ml = row.implied_win_home
            favorite = row.home
            underdog_implied_ml = row.implied_win_away
            underdog = row.away
        elif row.team_favorite_id == row.away:
            fav_implied_ml = row.implied_win_away
            favorite = row.away
            underdog_implied_ml = row.implied_win_home
            underdog = row.home

        # to prevent %/0 error
        if spread == 0:
            spread = -1

        fav_ev = round((win_dict[spread] * fav_implied_ml) - (1 - win_dict[spread]), 6)
        underdog_ev = round((1 - win_dict[spread] * underdog_implied_ml) - (win_dict[spread]), 6)

        #determine which has a create EV
        ev = 0
        if fav_ev >= underdog_ev:
            ev = fav_ev
            row['EV'] = ev
            row['Bet_Type'] = favorite
            
            
        else:
            ev = underdog_ev
            row['EV'] = ev
            row['Bet_Type'] = underdog
        # print(ev)
        if (ev >= min_ev) & (spread > largest_spread_fav):
            # print(ev)
            
            plus_ev = plus_ev.append(row)
    plus_ev = plus_ev.sort_values(by = 'EV', ascending=False)
    return plus_ev


In [106]:
ev_games = plus_ev_games(win_dict,df, .00 , largest_spread_fav=-20)
ev_games

,Bet_Type,EV,away,away_ML,home,home_ML,implied_win_away,implied_win_home,schedule_date,score_away,score_home,spread_favorite,team_favorite_id,total_implied_prob
3050,KC,0.960630,NYJ,1365.0,KC,-2440.0,0.068259,0.960630,2020-11-01,9.0,35.0,-19.5,KC,1.028889
2694,NE,0.938272,NE,-1520.0,MIA,975.0,0.938272,0.093023,2019-09-15,43.0,0.0,-18.0,NE,1.031295
2384,NE,0.933333,NYJ,887.5,NE,-1400.0,0.101266,0.933333,2017-12-31,6.0,26.0,-17.0,NE,1.034599
2603,LAC,0.928952,CIN,887.5,LAC,-1307.5,0.101266,0.928952,2018-12-09,21.0,26.0,-17.0,LAC,1.030218
2763,BUF,0.926335,MIA,870.0,BUF,-1257.5,0.103093,0.926335,2019-10-20,21.0,31.0,-17.0,BUF,1.029428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,PIT,0.061485,PIT,-130.0,CIN,110.0,0.565217,0.476190,2016-01-09,18.0,16.0,-3.0,PIT,1.041407
2179,TEN,0.056266,SEA,110.0,TEN,-126.0,0.476190,0.557522,2017-09-24,27.0,33.0,-3.0,TEN,1.033712
2313,ATL,0.053587,MIN,110.0,ATL,-124.0,0.476190,0.553571,2017-12-03,14.0,9.0,-3.0,ATL,1.029761
3053,SEA,0.037203,SF,-102.5,SEA,-112.5,0.506173,0.529412,2020-11-01,27.0,37.0,-3.0,SEA,1.035585


In [107]:
#simulate rough draft
ev_dates = ev_games[(ev_games['schedule_date'] > '2009-09-09') & (ev_games['schedule_date'] < '2020-03-09')]
ev_dates = ev_dates.sort_values(by = 'schedule_date')
ev_dates


,Bet_Type,EV,away,away_ML,home,home_ML,implied_win_away,implied_win_home,schedule_date,score_away,score_home,spread_favorite,team_favorite_id,total_implied_prob
2,BAL,0.830368,KC,572.5,BAL,-715.0,0.148699,0.877301,2009-09-13,24.0,38.0,-13.0,BAL,1.026000
3,CAR,0.155459,PHI,-137.5,CAR,117.5,0.578947,0.459770,2009-09-13,38.0,10.0,-2.5,PHI,1.038717
9,NO,0.871272,DET,680.0,NO,-870.0,0.128205,0.896907,2009-09-13,27.0,45.0,-14.0,NO,1.025112
17,PIT,0.076684,PIT,-142.5,CHI,125.0,0.587629,0.444444,2009-09-20,14.0,17.0,-3.0,PIT,1.032073
27,SEA,0.245857,SEA,100.0,SF,-115.5,0.500000,0.535963,2009-09-20,10.0,23.0,-1.0,SF,1.035963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932,KC,0.731075,HOU,383.0,KC,-480.5,0.207039,0.827735,2020-01-12,31.0,51.0,-10.0,KC,1.034774
2931,GB,0.326182,SEA,196.0,GB,-222.0,0.337838,0.689441,2020-01-12,23.0,28.0,-4.5,GB,1.027279
2933,KC,0.567426,TEN,290.5,KC,-335.5,0.256082,0.770379,2020-01-19,24.0,35.0,-7.0,KC,1.026461
2934,SF,0.580411,GB,307.5,SF,-372.5,0.245399,0.788360,2020-01-19,20.0,37.0,-8.0,SF,1.033759


In [110]:
#https://www.investopedia.com/terms/w/win-loss-ratio.asp
#w_l is a trader's number of winning trades relative to the number of losing trades.

#A negative Kelly criterion means that the bet is not favored by the model and should be avoided.

def kelly_criterion(win_dict, fav_spread, bet_type, team_favorite_id, win_loss):
    if(bet_type == team_favorite_id): 
        W = win_dict[fav_spread]
    elif fav_spread == 0:
        W = .5
    else:
        W = 1 - win_dict[fav_spread]
    R = sum(win_loss)/len(win_loss)
    K = W - ((1 - W)/(R))
    return K/100

win_loss = []
for x in range(0,60): #30 moving average start w/ 50% track record 
    if x % 2 == 0: 
        win_loss.append(0)
    else:
        win_loss.append(1)
def simple_simulation(plus_ev_df):
    start_amount = 1000
    #bet_amount = 25 #bet amount is from kelley criteria = W - ((1 - W)/R)
    for i, row in plus_ev_df.iterrows():
        kelly_percent = kelly_criterion(win_dict, row.spread_favorite, row.Bet_Type, row.team_favorite_id, win_loss)
        if kelly_percent < 0:
            continue #go to next game
        bet_amount = start_amount * kelly_percent
        print(bet_amount)
        if row.Bet_Type == row.home:
            win = (bet_amount * (1 + row.implied_win_home)) - bet_amount
            lose = bet_amount
            if row.score_home > row.score_away:
                win_loss.append(1) 
                win_loss.pop(0)#increase w/l ratio
                start_amount = start_amount + win
            elif row.score_home < row.score_away:
                start_amount = start_amount - lose
                win_loss.append(0)
                win_loss.pop(0)#increase w/l ratio

                
        elif row.Bet_Type == row.away:
            win = (bet_amount * (1 + row.implied_win_away)) - bet_amount
            lose = bet_amount
            if row.score_away > row.score_home:
                start_amount = start_amount + win
                win_loss.append(1)
                win_loss.pop(0)#increase w/l ratio
            elif row.score_away < row.score_home:
                start_amount = start_amount - lose
                win_loss.append(0)
                win_loss.pop(0)#increase w/l ratio

        print(start_amount)
    return start_amount


In [111]:
simple_simulation(ev_dates)




9.25
1008.11503425
9.681229807619433
1016.7981970330625
0.5620470113890872
1016.2361500216734
9.21029082852554
1024.3938335525702
0.5662455876826369
1023.8275879648876
9.334392996283714
1031.779109307558
10.31779109307558
1040.62293171845
9.451399242292911
1048.7241324191866
9.176344024098876
1056.4777852421332
0.9828134310145993
1055.4949718111186
9.64230939389113
1063.8073078280008
9.736423852579335
1071.9476007727314
10.719476007727314
1081.2185003901545
10.812185003901545
1090.2286509560108
9.308247166223047
1098.104861528699
9.078550568263903
1105.8864775984325
10.10087481638643
1114.7441670450426
9.249998465584156
1122.6727384798287
1.5918301914056248
1121.080908288423
1.5895731459894318
1122.0081587603925
1.5908878882519675
1122.949705075512
1.755014693573916
1124.0027138916562
1.7566604003553
1125.0118590785276
1.9127445706938067
1126.1358432578597
1.9146555680039368
1124.2211876898557
1.9114002716149296
1122.3097874182408
1.7540145909993197
1123.3390028379035
3.739492075012909

9044.996887329804

AttributeError: 'list' object has no attribute 'avg'